In [1]:
from cv2 import aruco
import cv2
import numpy as np
import matplotlib.pyplot as plt
import msgpack as mp
import msgpack_numpy as mpn
import os

from tqdm import tqdm
import csv
import pandas as pd
import polars as pl
import sys

sys.path.append('..')
sys.path.append(os.path.join(os.path.dirname(os.getcwd()), "frameworks", "dtag"))
import json

dtag_rep_pth = os.path.dirname(os.getcwd())
dtag_rep_pth = os.path.join(dtag_rep_pth, 'frameworks', "dtag")

In [2]:
from deeptag_model_setting import load_deeptag_models
from marker_dict_setting import load_marker_codebook
from stag_decode.detection_engine import DetectionEngine 

from stag_decode.pose_estimator import PoseSolver

In [3]:
_pth = os.getcwd()
_pth = os.path.dirname(_pth)
_parent_folder = "validation"
_calib_folder_name = "calibration_00"

_folder_name = "ar_30_5_random_00"
# _folder_name = "board"

_base_pth = os.path.join(_pth,"recorded_data",_parent_folder)

_webcam_calib_folder = os.path.join(_pth,"recorded_data",_parent_folder,_calib_folder_name)
_webcam_calib_pth = os.path.join( _webcam_calib_folder, "webcam_calibration.msgpack")

_webcam_8_point_pth = os.path.join(_webcam_calib_folder, "8p_dist_calib.msgpack")


with open(_webcam_calib_pth, "rb") as f:
    webcam_calib = mp.Unpacker(f, object_hook=mpn.decode)
    _temp = next(webcam_calib)
    _webcam_cam_mat = _temp[0]
    _webcam_dist = _temp[1]
    
with open(_webcam_8_point_pth, "rb") as f:
    webcam_8_point = mp.Unpacker(f, object_hook=mpn.decode)
    _temp = next(webcam_8_point)
    _webcam_8_point_mat = _temp[0]
    _webcam_8_point_dist = _temp[1]

ar_lframe_pth = os.path.join(_webcam_calib_folder, "webcam_rotmat_0.msgpack")
with open(ar_lframe_pth, "rb") as f:
    ar_lframe = mp.Unpacker(f, object_hook=mpn.decode)
    _ar_lframe_rot = next(ar_lframe)
    _ar_lframe_org = next(ar_lframe)

In [4]:
config_filename = f'{dtag_rep_pth}\\config_video.json'
config_dict = json.load(open(config_filename, 'r'))
cameraMatrix = _webcam_8_point_mat[0]
# cameraMatrix = config_dict['cameraMatrix']
distCoeffs = _webcam_8_point_dist[0][:8]
# distCoeffs = config_dict['distCoeffs']
tag_real_size_in_meter = config_dict['marker_size']
is_video = config_dict['is_video']!=0
filename = config_dict['filepath']
tag_family = config_dict['family']
codebook_filename  = config_dict['codebook'] if len(config_dict['codebook']) else os.path.join('codebook', tag_family + '_codebook.txt')
hamming_dist = config_dict['hamming_dist']
load_config_flag = True

In [5]:
load_model_flag = False
try:
    model_detector, model_decoder, device, tag_type, grid_size_cand_list = load_deeptag_models(tag_family, None) 
    load_model_flag = True
except:
    print('Cannot load models.')
    
# load marker library
load_codebook_flag = False
try:
    codebook = load_marker_codebook(os.path.join(dtag_rep_pth, codebook_filename), tag_type)
    load_codebook_flag = True
except:
    print('Cannot load codebook: %s'% codebook_filename)

===========> loading model <===========


In [6]:
stag_image_processor = DetectionEngine(model_detector, model_decoder, device, tag_type, grid_size_cand_list, 
            stg2_iter_num= 2, # 1 or 2
            min_center_score=0.2, min_corner_score = 0.2, # 0.1 or 0.2 or 0.3
            batch_size_stg2 = 4, # 1 or 2 or 4
            hamming_dist= hamming_dist, # 0, 2, 4
            cameraMatrix = cameraMatrix, distCoeffs=  distCoeffs, codebook = codebook,
            tag_real_size_in_meter_dict = {-1:tag_real_size_in_meter})

In [7]:
_video_pth = os.path.join(_base_pth,_folder_name,"webcam_color.msgpack")
_video_file = open(_video_pth, "rb")
_video_data = mp.Unpacker(_video_file, object_hook=mpn.decode)

default_ids = [12, 88, 89]

data = {"frame_id":[],"marker_ids":[], "corners":[], "tvec":[], "rvec":[]}
counter = 0

tags = []
for _frame in _video_data:
    decoded_tags = stag_image_processor.process(_frame, detect_scale=None)
    tags.append(decoded_tags)

>iter #0
>iter #1
0, >iter #0
>iter #1
0, >iter #0
>iter #1
0, >iter #0
>iter #1
0, >iter #0
>iter #1
0, >iter #0
>iter #1
0, >iter #0
>iter #1
0, >iter #0
>iter #1
0, >iter #0
>iter #1
0, >iter #0
>iter #1
0, >iter #0
>iter #1
0, >iter #0
>iter #1
0, >iter #0
>iter #1
0, >iter #0
>iter #1
0, >iter #0
>iter #1
0, >iter #0
>iter #1
0, >iter #0
>iter #1
0, >iter #0
>iter #1
0, >iter #0
>iter #1
0, >iter #0
>iter #1
0, >iter #0
>iter #1
0, >iter #0
>iter #1
0, >iter #0
>iter #1
0, >iter #0
>iter #1
0, >iter #0
>iter #1
0, >iter #0
>iter #1
0, >iter #0
>iter #1
0, >iter #0
>iter #1
0, >iter #0
>iter #1
0, >iter #0
>iter #1
0, >iter #0
>iter #1
0, >iter #0
>iter #1
0, >iter #0
>iter #1
0, >iter #0
>iter #1
0, >iter #0
>iter #1
0, >iter #0
>iter #1
0, >iter #0
>iter #1
0, >iter #0
>iter #1
0, >iter #0
>iter #1
0, >iter #0
>iter #1
0, >iter #0
>iter #1
0, >iter #0
>iter #1
0, >iter #0
>iter #1
0, >iter #0
>iter #1
0, >iter #0
>iter #1
0, >iter #0
>iter #1
0, >iter #0
>iter #1
0, >iter #0
>ite

In [11]:
tags[0][0]["tvecs"]

array([[ 0.14805999],
       [-0.12351333],
       [ 3.10779452]])